0115

지금가지 배운 내용을 기초로 tensorflow의 mnist example을 CNN으로 구현해 보자.

In [11]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

# 1. Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

# 전처리 단계(결측치, 이상치, 정규화, feature engineering, 기타 등등)
# mnist 예제에서는 이미 전처리가 끝난상태라서 따로 할게 없어

# 2. Model 정의(Tensorflow graph 생성)
tf.reset_default_graph()

# 2.1 placeholder
X = tf.placeholder(shape = [None,784], dtype=tf.float32)
Y = tf.placeholder(shape = [None,10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype=tf.float32)
# tensorflow버전에 따라서 dropout을 사용하는 속성이 변경
# 최신버전 => rate = 0.8 : drop시키는 노드의 비율
# 이전버전 => keep_prob = 0.2 : 살리는 노드의 비츌

# 2.2 Convoluion Layer
# 입력데이터는 4차원!
x_img = tf.reshape(X, [-1,28,28,1])  # 3차원

W1 = tf.Variable(tf.random_normal([3,3,1,32]))   # filter
L1 = tf.nn.conv2d(x_img,W1,strides=[1,1,1,1],   # 합성곱
                  padding = "SAME")
L1 = tf.nn.relu(L1)                             # Relu
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1],
                    padding = "SAME")          # pooling작업

W2 = tf.Variable(tf.random_normal([3,3,32,64]))
L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],
                  padding = "SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1],
                    padding = "SAME")


# 2.3 FC Layer : 학습이 시작되는부분
L2 = tf.reshape(L2,[-1,7*7*64])# 4차원을 2차원으로 reshape

W3 = tf.get_variable("weight3", shape = [7*7*64, 256],
                     initializer = tf.contrib.layers.xavier_initializer())   # 다음 layer에 몇개
b3 = tf.Variable(tf.random_normal([256]), name="bias3")
_layer1 = tf.nn.relu(tf.matmul(L2,W3)+b3)
layer1 = tf.nn.dropout(_layer1, keep_prob=dropout_rate)

W4 = tf.get_variable("weight4", shape = [256, 512],
                     initializer = tf.contrib.layers.xavier_initializer())   # 다음 layer에 몇개
b4 = tf.Variable(tf.random_normal([512]), name="bias4")
_layer2 = tf.nn.relu(tf.matmul(layer1,W4)+b4)
layer2 = tf.nn.dropout(_layer2, keep_prob=dropout_rate)

W5 = tf.get_variable("weight5", shape = [512, 10],
                     initializer = tf.contrib.layers.xavier_initializer())   # 다음 layer에 몇개
b5 = tf.Variable(tf.random_normal([10]), name="bias5")

logit = tf.matmul(layer2,W5) +b5
H = tf.nn.softmax(logit)

# cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                 labels=Y))
# train
train = tf.train.AdamOtimizer(learning_rate=0.01).minimize(cost)

# session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습

# 정확도
# prediction

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


### 이제 코드를 조금 더 나은 형태로 변경해보자

In [17]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

# 1. Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

# 2. Model 정의(Tensorflow graph 생성)
tf.reset_default_graph()

# 2.1 placeholder
X = tf.placeholder(shape = [None,784], dtype=tf.float32)
Y = tf.placeholder(shape = [None,10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype=tf.float32)

# 2.2 Convoluion Layer
# 입력데이터는 4차원!
x_img = tf.reshape(X, [-1,28,28,1])  # 3차원
L1 = tf.layers.conv2d(inputs = x_img, filters=32,
                      kernel_size=[3,3],
                      padding = "SAME",
                      strides = 1,
                      activation = tf.nn.relu)   # filter안쓰고 개수, 사이즈 알려줘야해 -> 함수의 인자로 넣어줌

'''
이 세가지 정보 함수에 다 넣음

W1 = tf.Variable(tf.random_normal([3,3,1,32]))   # filter
L1 = tf.nn.conv2d(x_img,W1,strides=[1,1,1,1],   # 합성곱
                  padding = "SAME")
L1 = tf.nn.relu(L1)                             # Relu
'''

L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],
                             padding="SAME",
                             strides=2)

'''
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1],
                    padding = "SAME")
'''
L1 = tf.layers.dropout(inputs=L1, rate=dropout_rate)


L2 = tf.layers.conv2d(inputs = x_img, filters=64,
                      kernel_size=[3,3],
                      padding = "SAME",
                      strides = 1,
                      activation = tf.nn.relu) 
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],
                             padding="SAME",
                             strides=2)
L2 = tf.layers.dropout(inputs=L2, rate=dropout_rate)

# 2.3 FC Layer => dense layer
L2 = tf.reshape(L2,[-1,7*7*64])# 4차원을 2차원으로 reshape

dense1 = tf.layers.dense(inputs=L2, units=256,
                         activation=tf.nn.relu)
dense1 = tf.layers.dropout(inputs=dense1, rate=dropout_rate)

dense2 = tf.layers.dense(inputs=dense1, units=128,
                         activation=tf.nn.relu)
dense2 = tf.layers.dropout(inputs=dense2, rate=dropout_rate)

dense3 = tf.layers.dense(inputs=dense2, units=512,
                         activation=tf.nn.relu)
dense3 = tf.layers.dropout(inputs=dense3, rate=dropout_rate)

# 몇개로 잡을 것인가만 정해주면돼

'''
W3 = tf.get_variable("weight3", shape = [7*7*64, 256],
                     initializer = tf.contrib.layers.xavier_initializer())   # 다음 layer에 몇개
b3 = tf.Variable(tf.random_normal([256]), name="bias3")
_layer1 = tf.nn.relu(tf.matmul(L2,W3)+b3)
'''

H = tf.layers.dense(inputs=dense3, units=10)    # 마지막에 relu안잡아줘

'''
알아서 xavier 초기법이나 최적의 초기법을 써줌, bias 안해줘도돼 다 포함됨

W5 = tf.get_variable("weight5", shape = [512, 10],
                     initializer = tf.contrib.layers.xavier_initializer())   # 다음 layer에 몇개
b5 = tf.Variable(tf.random_normal([10]), name="bias5")

logit = tf.matmul(layer2,W5) +b5
H = tf.nn.softmax(logit)
'''

# cost
cost = tf.losses.softmax_cross_entropy(Y,H)

'''
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                 labels=Y))
'''

# train
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
# session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습

# 정확도

# prediction

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


In [ ]:
# 런타임 -> GPU
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
import numpy as np

## 1. Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

## Model 정의 (Tensorflow graph 생성)
tf.reset_default_graph()  # 그래프 reset

# 2.1 placeholder
X = tf.placeholder(shape=[None,784], dtype=tf.float32)
Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
dropout_rate = tf.placeholder(dtype=tf.float32)  # 과적합을 피하기 위해서 nodes를 끈다

# 2.2 Convolution Layer
x_img = tf.reshape(X,[-1,28,28,1])  # 28*28, 색 1개, -1: 몇개의 이미지가 있는지는 알아서 계산해라 

L1 = tf.layers.conv2d(inputs=x_img, filters=32, kernel_size=[3,3], padding="SAME",
                     strides=1,activation=tf.nn.relu)  
L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2],padding="SAME", strides=2)
L1 = tf.layers.dropout(inputs=L1, rate=dropout_rate)

L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME",
                     strides=1, activation=tf.nn.relu) 
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2],padding="SAME", strides=2)
L2 = tf.layers.dropout(inputs=L2, rate=dropout_rate)

# 2.3 FC layer -> dense layer 
L2 = tf.reshape(L2, [-1,7*7*64])
dense1 = tf.layers.dense(inputs=L2, units=256, activation=tf.nn.relu)
dense1 = tf.nn.dropout(dense1, rate=dropout_rate)  

dense2 = tf.layers.dense(inputs=dense1, units=128, activation=tf.nn.relu)
dense2 = tf.nn.dropout(dense2, rate=dropout_rate)

dense3 = tf.layers.dense(inputs=dense2, units=512, activation=tf.nn.relu)
dense3 = tf.nn.dropout(dense3, rate=dropout_rate)

H = tf.layers.dense(inputs=dense3, units=10)  # H에서는 activation 쓰지 않음

# cost 
cost = tf.losses.softmax_cross_entropy(Y,H)

# train
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습 - epoch, batch 등 
num_of_epoch = 30
batch_size = 100

for step in range(num_of_epoch):
    num_of_iter = int(mnist.train.num_examples/batch_size)
    cost_val = 0
    for i in range(num_of_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)        
        _, cost_val = sess.run([train,cost],feed_dict = {X:batch_x,
                                                       Y:batch_y,
                                                       dropout_rate:0.3}) # 30% 끄고 학습해라
    if step%3==0:
        print("cost:{}".format(cost_val))

        
        tmp = sess.run(H, )

---
결국 우리의 MNIST예제는 multinomial 예제  
이미지 1개에 대한 예측값(H)이  
H의 도출값은  
[0.5, 0.3, 0.2, 0.001, 0.99, 0.44, ...]  
1이 될 확률, 2가 될 확률...  


**ensemble**은 모델을 여러개 만들어요  
이미지 1개에 대한 각 모델의 예측값  
H1 => [0.5, 0.3, 0.2, 0.001, 0.99, 0.44, ...]    
H2 => [0.4, 0.2, 0.3, 0.011, 0.90, 0.64, ...]  
H3 => [0.8, 0.3, 0.4, 0.05, 0.78, 0.34, ...]    
Hn => [...]  

SUM => [1.7, 0.8, 0.9, 0.62, ...]  
학습이 잘된 모델에 대해서 다 물어봐   
여러사람의 의견 모아서 가장 좋은 대안을 내놓는다고 보면된다.  
한사람의 전문가한테 의견을 구하는게 아니라 여러사람의 전문가에게 의견을 구해서   
가장 그럴 듯한 대답을 찾아내는 방식


프로그래밍의 영역